# Overview of Denver crime files
- Consists of reported crimes to the Denver Police Department based on the National Incident Based Reporting System (NIBRS)
- data was separated into two files; 2014-2019, 2018-2023, which will be merged at the end for EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2014-2019

In [24]:
df_raw = pd.read_pickle('/content/drive/My Drive/ColabNotebooks/Denver_2014_2019.pkl')
df_raw.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [25]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494491 entries, 0 to 494490
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   INCIDENT_ID             494491 non-null  int64  
 1   OFFENSE_ID              494491 non-null  int64  
 2   OFFENSE_CODE            494491 non-null  int64  
 3   OFFENSE_CODE_EXTENSION  494491 non-null  int64  
 4   OFFENSE_TYPE_ID         494491 non-null  object 
 5   OFFENSE_CATEGORY_ID     494491 non-null  object 
 6   FIRST_OCCURRENCE_DATE   494491 non-null  object 
 7   LAST_OCCURRENCE_DATE    156315 non-null  object 
 8   REPORTED_DATE           494491 non-null  object 
 9   INCIDENT_ADDRESS        445994 non-null  object 
 10  GEO_X                   490419 non-null  float64
 11  GEO_Y                   490419 non-null  float64
 12  GEO_LON                 490419 non-null  float64
 13  GEO_LAT                 490419 non-null  float64
 14  DISTRICT_ID         

In [26]:
df_raw.dropna(subset='GEO_LON', inplace=True)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490419 entries, 0 to 494490
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   INCIDENT_ID             490419 non-null  int64  
 1   OFFENSE_ID              490419 non-null  int64  
 2   OFFENSE_CODE            490419 non-null  int64  
 3   OFFENSE_CODE_EXTENSION  490419 non-null  int64  
 4   OFFENSE_TYPE_ID         490419 non-null  object 
 5   OFFENSE_CATEGORY_ID     490419 non-null  object 
 6   FIRST_OCCURRENCE_DATE   490419 non-null  object 
 7   LAST_OCCURRENCE_DATE    153944 non-null  object 
 8   REPORTED_DATE           490419 non-null  object 
 9   INCIDENT_ADDRESS        445994 non-null  object 
 10  GEO_X                   490419 non-null  float64
 11  GEO_Y                   490419 non-null  float64
 12  GEO_LON                 490419 non-null  float64
 13  GEO_LAT                 490419 non-null  float64
 14  DISTRICT_ID         

## Filter out crimes not near universities

In [27]:
# Universities coordinates
coords = {
    ('Rocky Mountain College of Art and Design', 39.74335, -105.06924),
    ('Colorado Christian University', 39.7126, -105.0960),
    ('Colorado School of Mines', 39.7510, -105.2226),
    ('University Of Denver', 39.6766, -104.9619),
    ('Regis University', 39.7896, -105.0302),
    ('University of Colorado Denver', 39.7464, -105.0023),
    ('Metropolitan State University of Denver', 39.7439, -105.0047),
    ('University of Colorado', 40.0073, -105.2660)
}

In [28]:
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree
import folium
import plotly.graph_objects as go

universities_df = pd.DataFrame(coords, columns=['University', 'Latitude', 'Longitude'])
universities_gdf = gpd.GeoDataFrame(universities_df, geometry=[Point(xy) for xy in zip(universities_df.Longitude, universities_df.Latitude)])

crime_gdf = gpd.GeoDataFrame(df_raw.drop(['LAST_OCCURRENCE_DATE', 'INCIDENT_ADDRESS'], axis=1), geometry=gpd.points_from_xy(df_raw['GEO_LON'], df_raw['GEO_LAT']))

# Build a spatial index for efficient nearest-neighbor queries
tree = BallTree(universities_gdf[['Longitude', 'Latitude']].values, metric='haversine')

# Query the spatial index to find the distance to the nearest university for each crime
distances, indices = tree.query(crime_gdf[['GEO_LON', 'GEO_LAT']].values, return_distance=True)

# Convert distances to meters (BallTree returns distances in radians)
distances_meters = distances * 6371000

# Determine whether each crime is near a university
near_university = distances_meters <= 3000 # for example, using 4km as the threshold

# Filter the crimes that are near universities
crimes_near_universities = crime_gdf[near_university]

# Filter the crimes that are not near any university by metric
crimes_not_near_universities = crime_gdf[~near_university]

In [29]:
# Calculate average latitude and longitude for map centering
average_lat, average_lon = universities_gdf['Latitude'].mean(), universities_gdf['Longitude'].mean()

# Create a base map
m = folium.Map(location=[average_lat, average_lon], zoom_start=13)

# Add points for crimes and universities
for idx, row in crimes_near_universities.iterrows():
    folium.Marker([row['GEO_LAT'], row['GEO_LON']], icon=folium.Icon(color='red')).add_to(m)
for idx, row in universities_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], icon=folium.Icon(color='blue')).add_to(m)

# Display the map
m

In [30]:
crimes_near_universities['GEO_LON'].value_counts()

-105.004669    165
-105.002489    154
-105.002030    111
-105.002627     39
-105.002283     37
-105.004869     29
-104.961778     27
-105.002551     23
-105.001992     10
-104.962105      6
-105.004771      4
-105.004511      2
-105.002414      1
-105.004982      1
-105.002023      1
-105.002024      1
-105.001971      1
-105.002363      1
Name: GEO_LON, dtype: int64

In [31]:
crimes_near_universities.shape

(613, 18)

In [32]:
crimes_near_universities['datetime'] = pd.to_datetime(crimes_near_universities['FIRST_OCCURRENCE_DATE'])
crimes_near_universities[crimes_near_universities['datetime']>="2018-01-01"].sort_values('datetime', ascending=True)

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,REPORTED_DATE,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,geometry,datetime
18736,201825852,201825852540100,5401,0,traffic-accident-hit-and-run,traffic-accident,1/11/2018 5:30:00 PM,1/11/2018 6:08:00 PM,3139895.0,1697596.0,-105.002489,39.747654,1,123,auraria,0,1,POINT (-105.00249 39.74765),2018-01-11 17:30:00
2176,201831422,201831422570700,5707,0,criminal-trespassing,all-other-crimes,1/13/2018 11:48:00 PM,1/13/2018 11:48:00 PM,3139956.0,1697085.0,-105.002283,39.746251,1,123,auraria,1,0,POINT (-105.00228 39.74625),2018-01-13 23:48:00
1818,201857383,201857383544100,5441,0,traffic-accident,traffic-accident,1/24/2018 4:21:00 PM,1/24/2018 5:47:00 PM,3139895.0,1697596.0,-105.002489,39.747654,1,123,auraria,0,1,POINT (-105.00249 39.74765),2018-01-24 16:21:00
215068,201859624,201859624544100,5441,0,traffic-accident,traffic-accident,1/25/2018 2:28:00 PM,1/25/2018 3:04:00 PM,3139895.0,1697596.0,-105.002489,39.747654,1,123,auraria,0,1,POINT (-105.00249 39.74765),2018-01-25 14:28:00
428,201870691,201870691570700,5707,0,criminal-trespassing,all-other-crimes,1/30/2018 8:10:00 AM,1/30/2018 10:09:00 AM,3139895.0,1697596.0,-105.002489,39.747654,1,123,auraria,1,0,POINT (-105.00249 39.74765),2018-01-30 08:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47209,2019306348,2019306348570700,5707,0,criminal-trespassing,all-other-crimes,5/16/2019 1:24:00 PM,5/16/2019 1:24:00 PM,3139956.0,1697085.0,-105.002283,39.746251,1,123,auraria,1,0,POINT (-105.00228 39.74625),2019-05-16 13:24:00
48594,2019362636,2019362636239900,2399,0,theft-other,larceny,6/10/2019 3:40:00 PM,6/10/2019 4:20:00 PM,3140028.0,1696884.0,-105.002030,39.745698,1,123,auraria,1,0,POINT (-105.00203 39.74570),2019-06-10 15:40:00
46938,2019365156,2019365156239901,2399,1,theft-bicycle,larceny,6/11/2019 9:30:00 AM,6/11/2019 4:56:00 PM,3140028.0,1696884.0,-105.002030,39.745698,1,123,auraria,1,0,POINT (-105.00203 39.74570),2019-06-11 09:30:00
46573,2019377630,2019377630544100,5441,0,traffic-accident,traffic-accident,6/17/2019 1:30:00 AM,6/17/2019 1:30:00 AM,3139895.0,1697596.0,-105.002489,39.747654,1,123,auraria,0,1,POINT (-105.00249 39.74765),2019-06-17 01:30:00


# 2018-2023

In [33]:
df_raw2 = pd.read_pickle('/content/drive/My Drive/ColabNotebooks/Denver_2018_2023.pkl')
df_raw2.head()

,incident_id,offense_id,offense_code,offense_code_extension,offense_type_id,offense_category_id,first_occurrence_date,last_occurrence_date,reported_date,incident_address,geo_x,geo_y,geo_lon,geo_lat,district_id,precinct_id,neighborhood_id,is_crime,is_traffic,victim_count
0,202268791,202268791299900,2999,0,criminal-mischief-other,public-disorder,2/10/2022 2:50:00 AM,NaN,2/10/2022 3:16:00 AM,1107 N SANTA FE DR,3140929.0,1692612.0,-104.998910,39.733957,1,123,lincoln-park,1,0,1
1,2021387586,2021387586299900,2999,0,criminal-mischief-other,public-disorder,7/7/2021 9:02:00 PM,NaN,7/8/2021 12:55:00 AM,815 16TH ST,3142470.0,1697098.0,-104.993342,39.746248,6,611,cbd,1,0,1
2,2020641486,2020641486299900,2999,0,criminal-mischief-other,public-disorder,10/29/2020 1:30:00 AM,NaN,10/29/2020 4:31:00 AM,4745 N FEDERAL BLVD,3133352.0,1710396.0,-105.025520,39.782888,1,111,berkeley,1,0,1
3,2018612468,2018612468299900,2999,0,criminal-mischief-other,public-disorder,9/6/2018 5:00:00 PM,9/6/2018 11:00:00 PM,9/7/2018 9:58:00 AM,65 S FEDERAL BLVD,3133534.0,1685797.0,-105.025330,39.715357,4,411,barnum,1,0,1
4,2020293614,2020293614299900,2999,0,criminal-mischief-other,public-disorder,5/8/2020 5:00:00 AM,5/8/2020 6:30:00 PM,5/13/2020 10:00:00 AM,12295 E ALBROOK DR,3184065.0,1710782.0,-104.845074,39.783082,5,521,montbello,1,0,1


In [34]:
df_raw2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386865 entries, 0 to 386864
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   incident_id             386865 non-null  int64  
 1   offense_id              386865 non-null  int64  
 2   offense_code            386865 non-null  int64  
 3   offense_code_extension  386865 non-null  int64  
 4   offense_type_id         386865 non-null  object 
 5   offense_category_id     386865 non-null  object 
 6   first_occurrence_date   386865 non-null  object 
 7   last_occurrence_date    211309 non-null  object 
 8   reported_date           386865 non-null  object 
 9   incident_address        371362 non-null  object 
 10  geo_x                   371362 non-null  float64
 11  geo_y                   371362 non-null  float64
 12  geo_lon                 371096 non-null  float64
 13  geo_lat                 371096 non-null  float64
 14  district_id         

In [35]:
df_raw2.dropna(subset='geo_lon', inplace=True)
df_raw2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371096 entries, 0 to 386864
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   incident_id             371096 non-null  int64  
 1   offense_id              371096 non-null  int64  
 2   offense_code            371096 non-null  int64  
 3   offense_code_extension  371096 non-null  int64  
 4   offense_type_id         371096 non-null  object 
 5   offense_category_id     371096 non-null  object 
 6   first_occurrence_date   371096 non-null  object 
 7   last_occurrence_date    206681 non-null  object 
 8   reported_date           371096 non-null  object 
 9   incident_address        371096 non-null  object 
 10  geo_x                   371096 non-null  float64
 11  geo_y                   371096 non-null  float64
 12  geo_lon                 371096 non-null  float64
 13  geo_lat                 371096 non-null  float64
 14  district_id         

## Filter out crimes not near universities

In [36]:
universities_df = pd.DataFrame(coords, columns=['University', 'Latitude', 'Longitude'])
universities_gdf = gpd.GeoDataFrame(universities_df, geometry=[Point(xy) for xy in zip(universities_df.Longitude, universities_df.Latitude)])

crime_gdf2 = gpd.GeoDataFrame(df_raw2.drop(['last_occurrence_date', 'district_id', 'district_id', 'precinct_id', 'neighborhood_id'], axis=1), geometry=gpd.points_from_xy(df_raw2['geo_lon'], df_raw2['geo_lat']))

# Build a spatial index for efficient nearest-neighbor queries
tree = BallTree(universities_gdf[['Longitude', 'Latitude']].values, metric='haversine')

# Query the spatial index to find the distance to the nearest university for each crime
distances, indices = tree.query(crime_gdf2[['geo_lon', 'geo_lat']].values, return_distance=True)

# Convert distances to meters (BallTree returns distances in radians)
distances_meters = distances * 6371000

# Determine whether each crime is near a university
near_university = distances_meters <= 3000 # for example, using 4km as the threshold

# Filter the crimes that are near universities
crimes_near_universities2 = crime_gdf2[near_university]

# Filter the crimes that are not near any university by metric
crimes_not_near_universities2 = crime_gdf2[~near_university]

In [37]:
# Calculate average latitude and longitude for map centering
average_lat, average_lon = universities_gdf['Latitude'].mean(), universities_gdf['Longitude'].mean()

# Create a base map
m = folium.Map(location=[average_lat, average_lon], zoom_start=13)

# Add points for crimes and universities
for idx, row in crimes_near_universities2.iterrows():
    folium.Marker([row['geo_lat'], row['geo_lon']], icon=folium.Icon(color='red')).add_to(m)
for idx, row in universities_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], icon=folium.Icon(color='blue')).add_to(m)

# Display the map
m

In [38]:
crimes_near_universities2['geo_lon'].value_counts()

-105.002030    143
-105.002283     44
-105.002489     29
-105.004869     20
-104.961778      6
-105.002627      5
-105.001992      3
-105.004511      2
-104.962105      2
-105.001971      2
-105.004406      1
-105.002414      1
-105.002416      1
-105.004863      1
Name: geo_lon, dtype: int64

# Check for duplicates between both files

In [39]:
crimes_near_universities2['datetime'] = pd.to_datetime(crimes_near_universities2['first_occurrence_date'])
# crimes_near_universities2.sort_values('datetime', ascending=True)
crimes_near_universities['datetime'].describe()

count                     613
unique                    559
top       2016-11-17 14:47:00
freq                        4
first     2014-01-05 09:50:00
last      2019-06-28 09:15:00
Name: datetime, dtype: object

In [40]:
crimes_near_universities[crimes_near_universities['datetime']>="2018-01-01"].sort_values('datetime', ascending=True)['datetime'].describe()

count                     164
unique                    149
top       2018-07-01 04:15:00
freq                        3
first     2018-01-11 17:30:00
last      2019-06-28 09:15:00
Name: datetime, dtype: object

In [41]:
set1 = set(crimes_near_universities[crimes_near_universities['datetime']>="2018-01-01"]['OFFENSE_ID'].values)
set2 = set(crimes_near_universities2['offense_id'].values)
intersect_list = (set2.intersection(set1))

In [42]:
crimes_near_universities2[crimes_near_universities2['offense_id'].isin(set2.intersection(set1))].sort_values('offense_id')['offense_category_id'].values == crimes_near_universities[crimes_near_universities['OFFENSE_ID'].isin(set2.intersection(set1))].sort_values('OFFENSE_ID')['OFFENSE_CATEGORY_ID'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

Hence, intersecting crimes (n=95) is the same
- just need to merge non-intersecting crimes

# Merging of files for 2014-2023 data
1. merge all non-intersected files based on offense ID which is unique
2. remap offense code to align with 2018-2023's mapping

In [43]:
display(crimes_near_universities.shape)
crimes_near_universities['OFFENSE_ID'].nunique()

(613, 19)

613

In [44]:
display(crimes_near_universities2.shape)
crimes_near_universities2['offense_id'].nunique()

(260, 18)

260

In [45]:
crimes_near_universities.rename(columns={'OFFENSE_ID':'offense_id', 'OFFENSE_CODE':'offense_code', 'OFFENSE_TYPE_ID':'offense_type_id', 'OFFENSE_CATEGORY_ID':'offense_category_id', 'GEO_LON':'geo_lon', 'GEO_LAT':'geo_lat'}, inplace=True)

In [46]:
# remove duplicated offense_id from 2014-2019 data
crimes_near_universities = crimes_near_universities[crimes_near_universities['offense_id'].isin(intersect_list)==False]

# common columns
common_cols = ['offense_id', 'datetime', 'offense_code', 'offense_type_id', 'offense_category_id', 'geo_lon', 'geo_lat']

# merge
merged_df = crimes_near_universities[common_cols].append(crimes_near_universities2[common_cols])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 778 entries, 285 to 384407
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   offense_id           778 non-null    int64         
 1   datetime             778 non-null    datetime64[ns]
 2   offense_code         778 non-null    int64         
 3   offense_type_id      778 non-null    object        
 4   offense_category_id  778 non-null    object        
 5   geo_lon              778 non-null    float64       
 6   geo_lat              778 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 48.6+ KB


In [47]:
sum(merged_df['offense_id'].duplicated())

0

In [48]:
sorted(dict(zip(crimes_near_universities['offense_code'], crimes_near_universities['offense_category_id'])).items())

[(1202, 'robbery'),
 (1313, 'other-crimes-against-persons'),
 (1315, 'aggravated-assault'),
 (1316, 'public-disorder'),
 (2203, 'burglary'),
 (2205, 'burglary'),
 (2206, 'burglary'),
 (2303, 'larceny'),
 (2305, 'theft-from-motor-vehicle'),
 (2308, 'larceny'),
 (2399, 'larceny'),
 (2404, 'auto-theft'),
 (2501, 'white-collar-crime'),
 (2503, 'white-collar-crime'),
 (2606, 'white-collar-crime'),
 (2999, 'public-disorder'),
 (3512, 'drug-alcohol'),
 (3532, 'drug-alcohol'),
 (3542, 'drug-alcohol'),
 (3550, 'drug-alcohol'),
 (3562, 'drug-alcohol'),
 (3572, 'drug-alcohol'),
 (3605, 'other-crimes-against-persons'),
 (4104, 'drug-alcohol'),
 (4801, 'all-other-crimes'),
 (4803, 'all-other-crimes'),
 (4899, 'all-other-crimes'),
 (5016, 'all-other-crimes'),
 (5203, 'all-other-crimes'),
 (5212, 'all-other-crimes'),
 (5309, 'public-disorder'),
 (5311, 'all-other-crimes'),
 (5312, 'public-disorder'),
 (5313, 'public-disorder'),
 (5399, 'public-disorder'),
 (5401, 'traffic-accident'),
 (5420, 'traffic

In [49]:
sorted(dict(zip(crimes_near_universities2['offense_code'], crimes_near_universities2['offense_category_id'])).items())

[(1313, 'other-crimes-against-persons'),
 (1315, 'aggravated-assault'),
 (1316, 'public-disorder'),
 (2005, 'arson'),
 (2203, 'burglary'),
 (2205, 'burglary'),
 (2206, 'burglary'),
 (2302, 'larceny'),
 (2304, 'theft-from-motor-vehicle'),
 (2305, 'theft-from-motor-vehicle'),
 (2308, 'larceny'),
 (2399, 'larceny'),
 (2404, 'auto-theft'),
 (2589, 'white-collar-crime'),
 (2604, 'white-collar-crime'),
 (2607, 'white-collar-crime'),
 (2609, 'white-collar-crime'),
 (2999, 'public-disorder'),
 (3530, 'drug-alcohol'),
 (3532, 'drug-alcohol'),
 (3572, 'drug-alcohol'),
 (3599, 'drug-alcohol'),
 (4801, 'all-other-crimes'),
 (4803, 'all-other-crimes'),
 (4899, 'all-other-crimes'),
 (4901, 'all-other-crimes'),
 (5016, 'all-other-crimes'),
 (5212, 'all-other-crimes'),
 (5213, 'all-other-crimes'),
 (5299, 'all-other-crimes'),
 (5309, 'public-disorder'),
 (5311, 'all-other-crimes'),
 (5312, 'public-disorder'),
 (5707, 'all-other-crimes'),
 (7399, 'all-other-crimes')]

surprisingly, 2018-2023 does not have any traffic accidents near universities

In [50]:
merged_df['offense_category_id'].value_counts()

all-other-crimes                256
larceny                         210
traffic-accident                128
public-disorder                  80
burglary                         39
drug-alcohol                     25
other-crimes-against-persons     14
white-collar-crime                9
aggravated-assault                7
theft-from-motor-vehicle          5
auto-theft                        3
robbery                           1
arson                             1
Name: offense_category_id, dtype: int64

# Extract crime types with high counts from 'all-other-crimes'
- too signicant to be generalised

In [51]:
merged_df[merged_df['offense_category_id']=='all-other-crimes']['offense_type_id'].value_counts()

criminal-trespassing              150
traf-other                         32
violation-of-court-order           14
public-order-crimes-other          13
police-false-information           12
police-interference                 9
violation-of-restraining-order      5
weapon-poss-illegal-dangerous       4
public-fighting                     4
police-resisting-arrest             3
traf-habitual-offender              2
weapon-unlawful-discharge-of        2
weapon-other-viol                   2
weapon-carrying-prohibited          1
vehicular-eluding-no-chase          1
escape                              1
weapon-by-prev-offender-powpo       1
Name: offense_type_id, dtype: int64

In [52]:
mask = (merged_df['offense_type_id'] == 'criminal-trespassing')
merged_df.loc[mask, 'offense_category_id'] = 'Trespassing'

In [53]:
merged_df['offense_category_id'].value_counts()

larceny                         210
Trespassing                     150
traffic-accident                128
all-other-crimes                106
public-disorder                  80
burglary                         39
drug-alcohol                     25
other-crimes-against-persons     14
white-collar-crime                9
aggravated-assault                7
theft-from-motor-vehicle          5
auto-theft                        3
robbery                           1
arson                             1
Name: offense_category_id, dtype: int64

In [54]:
merged_df.reset_index(drop=True, inplace=True)

In [55]:
# merged_df.to_csv('/content/drive/My Drive/ColabNotebooks/Denver_crimes_near_universities_2014_2023.csv', index=False)

In [56]:
# df_raw2.to_pickle('/content/drive/My Drive/ColabNotebooks/Denver_2018_2023.pkl')